Final compilation which gives result of the extracted on getting the date and symbol


In [2]:
!pip install sec-edgar-downloader requests beautifulsoup4 pandas lxml

In [3]:
import requests
import re
import shutil
import os


def get_cik(ticker):
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?CIK={ticker}&owner=exclude&action=getcompany&output=atom"
    headers = {
        'User-Agent': 'abc@gmail.com'  # Replace with your information
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # print(response.text)
        cik_search = re.search(r'<cik>(\d+)</cik>', response.text)
        if cik_search:
            return cik_search.group(1) # Remove leading zeros

    print(f"Could not find CIK for ticker {ticker}")
    return None

def remove_hifen(s):
    temp = s;
    return temp.replace("-", "")


def get_name(temp):
  x = temp.split("=")
  # print(x[1])

  res=""
  j=1
  while True:
    if x[1][j] == '"':
      break
    res+=x[1][j]
    j+=1
  # print(res)

  return res


def get_modified_url(url , res ):
  br = url.split("/")
  br.pop()
  br.append(res)
  new_url = "/".join(br)
  return new_url

def delete_folder(folder_path):
  if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' and its contents have been deleted.")
  else:
      print(f"Folder '{folder_path}' does not exist.")

In [4]:
from ast import YieldFrom
from sec_edgar_downloader import Downloader
import os
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Initialize the downloader with company name and email address
dl = Downloader("Your Company Name", "your.email@example.com")

# Example tickers to download SEC filings for
# tickers = [stock]  # Replace with your dataset's tickers

def download_data(stock , date ):
  [m,d,y] = date.split("/")
  # print(m, d, y)
  m = int(m)

  start_date = "2015-08-01"  # Start date (YYYY-MM-DD)
  end_date = "2016-08-01"

  filing_type = "";

  if m>=1 and m<=3:
    filing_type = "10-Q"

  elif m>=4 and m<=6:
    filing_type = "10-Q"

  elif m>=7 and m<=9:
    filing_type = "10-Q"
  else:
    filing_type = "10-K"


  start_date = f"{y}-{str(m)}-{d}"
  end_date = f"{str(int(y)+1)}-{str(m)}-{d}"

  start_date = str(start_date)
  end_date = str(end_date)

  # print(start_date , end_date)

  #  # Define the date range for filtering
  #  # End date (YYYY-MM-DD)

  # start_date = "2023-01-01"  # Start date (YYYY-MM-DD)
  # end_date = "2024-04-01"


  # print(start_date , end_date)
  # print(stock , filing_type)

  # Loop through each ticker and filing type
  # for ticker in tickers:]
  try:
    dl.get(filing_type, stock , after=start_date, before=end_date)
  except Exception as e:
    print(e)
  # print("Data Downloaded successfully")
  return "done"

In [5]:
import re

def clean_xbrl(file_content):
    def xbrl_clean(cond1, cond2, str0):
        locations = [0]
        placement1 = [m.start() for m in re.finditer(cond1, str0.lower())]
        if placement1:
            placement2 = [m.end() for m in re.finditer(cond2, str0.lower())]
            len1 = len(placement1)
            placement1.append(len(str0))
            for i in range(len1):
                placement3 = [j for j in placement2 if placement1[i] < j < placement1[i + 1]]
                locations.append(placement1[i])
                if placement3:
                    locations.append(placement3[0])
                else:
                    locations.append(placement1[i])
        return locations

    # Apply XBRL cleaning steps
    locations_xbrl = xbrl_clean("<xbrl", "</xbrl.*>", file_content)
    locations_xbrl.append(len(file_content))

    clean_content = ""
    if len(locations_xbrl) % 2 == 0:
        for i in range(0, len(locations_xbrl), 2):
            clean_content += file_content[locations_xbrl[i]:locations_xbrl[i + 1]]

    # Replace HTML entities with spaces or appropriate characters
    patterns = [
        (r'&nbsp;', ' '),
        (r'&lt;', '<'),
        (r'&gt;', '>'),
        (r'&amp;', '&'),
        (r'&apos;', "'"),
        (r'&quot;', '"'),
        (r'&#\d{1,5};', ''),
        (r'&#.{1,5};', ''),
    ]
    for pattern, replacement in patterns:
        clean_content = re.sub(pattern, replacement, clean_content)

    # Replace tags with spaces to ensure spacing between words
    tag_patterns = [
        r'<div.*?>', r'</div>',
        r'<tr.*?>', r'</tr>',
        r'<td.*?>', r'</td>',
        r'<font.*?>', r'</font>',
        r'<p.*?>', r'</p>',
        r'<span.*?>', r'</span>',
        r'<br.*?>', r'<hr.*?>'
    ]
    for pattern in tag_patterns:
        clean_content = re.sub(pattern, ' ', clean_content)

    # Remove remaining tags and clean up spaces
    clean_content = re.sub(r'<.*?>', ' ', clean_content)
    clean_content = clean_content.replace('\r\n', ' ').replace('\n', ' ')
    clean_content = re.sub(r'\s{2,}', ' ', clean_content).strip()

    return clean_content



In [26]:
import re

def extract_mda(content, filing_type):
    # Define the regex patterns for Item 3 (10-Q) and Item 7 (10-K)
    item2_patterns = [
    r"item 2\. managements discussion and analysis",
    r"item 2\.managements discussion and analysis",
    r"item2\. managements discussion and analysis",
    r"item2\.managements discussion and analysis",
    r"item 2\. management discussion and analysis",
    r"item 2\.management discussion and analysis",
    r"item2\. management discussion and analysis",
    r"item2\.management discussion and analysis",
    r"item 2 managements discussion and analysis",
    r"item 2managements discussion and analysis",
    r"item2 managements discussion and analysis",
    r"item2managements discussion and analysis",
    r"item 2 management discussion and analysis",
    r"item 2management discussion and analysis",
    r"item2 management discussion and analysis",
    r"item2management discussion and analysis",
    r"item 2: managements discussion and analysis",
    r"item 2:managements discussion and analysis",
    r"item2: managements discussion and analysis",
    r"item2:managements discussion and analysis",
    r"item 2: management discussion and analysis",
    r"item 2:management discussion and analysis",
    r"item2: management discussion and analysis",
    r"item2:management discussion and analysis",
    r"item 2\. management's discussion and analysis",
    r"item 2\.management's discussion and analysis",
    r"item2\. management's discussion and analysis",
    r"item2\.management's discussion and analysis",
    r"item 2 management's discussion and analysis",
    r"item 2management's discussion and analysis",
    r"item2 management's discussion and analysis",
    r"item2management's discussion and analysis",
    r"item 2: management's discussion and analysis",
    r"item 2:management's discussion and analysis",
    r"item2: management's discussion and analysis",
    r"item2:management's discussion and analysis",
    r"item 2. Management&rsquo;s Discussion and Analysis"
]

    end_pattern_3q = [
    r"item 3\. quantitative and qualitative disclosures about market risk",
    r"item 3\.quantitative and qualitative disclosures about market risk",
    r"item3\. quantitative and qualitative disclosures about market risk",
    r"item3\.quantitative and qualitative disclosures about market risk",
    r"item 3 quantitative and qualitative disclosures about market risk",
    r"item 3quantitative and qualitative disclosures about market risk",
    r"item3 quantitative and qualitative disclosures about market risk",
    r"item3quantitative and qualitative disclosures about market risk",
    r"item 3: quantitative and qualitative disclosures about market risk",
    r"item 3:quantitative and qualitative disclosures about market risk",
    r"item3: quantitative and qualitative disclosures about market risk",
    r"item3:quantitative and qualitative disclosures about market risk",
    r"part ii other information",
    r"part ii: other information",
    r"part ii otherinformation",
    r"part ii:other information",
    r"part ii:otherinformation",
]

    item7_patterns = [
        r"item 7\. managements discussion and analysis",
        r"item 7\.managements discussion and analysis",
        r"item7\. managements discussion and analysis",
        r"item7\.managements discussion and analysis",
        r"item 7\. management discussion and analysis",
        r"item 7\.management discussion and analysis",
        r"item7\. management discussion and analysis",
        r"item7\.management discussion and analysis",
        r"item 7 managements discussion and analysis",
        r"item 7managements discussion and analysis",
        r"item7 managements discussion and analysis",
        r"item7managements discussion and analysis",
        r"item 7 management discussion and analysis",
        r"item 7management discussion and analysis",
        r"item7 management discussion and analysis",
        r"item7management discussion and analysis",
        r"item 7: managements discussion and analysis",
        r"item 7:managements discussion and analysis",
        r"item7: managements discussion and analysis",
        r"item7:managements discussion and analysis",
        r"item 7: management discussion and analysis",
        r"item 7:management discussion and analysis",
        r"item7: management discussion and analysis",
        r"item7:management discussion and analysis",

        r"item 7\. management's discussion and analysis",
        r"item 7\.management's discussion and analysis",
        r"item7\. management's discussion and analysis",
        r"item7\.management's discussion and analysis",
        r"item 7\. management's discussion and analysis",
        r"item 7\.management's discussion and analysis",
        r"item7\. management's discussion and analysis",
        r"item7\.management's discussion and analysis",
        r"item 7 management's discussion and analysis",
        r"item 7management's discussion and analysis",
        r"item7 management's discussion and analysis",
        r"item7management's discussion and analysis",
        r"item 7 management's discussion and analysis",
        r"item 7management's discussion and analysis",
        r"ITEM7. MANAGEMENTS DISCUSSION AND ANALYSIS OF FINANCIAL"
    ]

    item7a_patterns = [
    r"item 7a\. quantitative and qualitative disclosures about market risk",
    r"item 7a\.quantitative and qualitative disclosures about market risk",
    r"item7a\. quantitative and qualitative disclosures about market risk",
    r"item7a\.quantitative and qualitative disclosures about market risk",
    r"item 7a quantitative and qualitative disclosures about market risk",
    r"item 7aquantitative and qualitative disclosures about market risk",
    r"item7a quantitative and qualitative disclosures about market risk",
    r"item7aquantitative and qualitative disclosures about market risk",
    r"item 7a: quantitative and qualitative disclosures about market risk",
    r"item 7a:quantitative and qualitative disclosures about market risk",
    r"item7a: quantitative and qualitative disclosures about market risk",
    r"item7a:quantitative and qualitative disclosures about market risk",
    r"item 8\. financial statements and supplementary data",
    r"item 8\.financial statements and supplementary data",
    r"item8\. financial statements and supplementary data",
    r"item8\.financial statements and supplementary data",
    r"item 8 financial statements and supplementary data",
    r"item 8financial statements and supplementary data",
    r"item8 financial statements and supplementary data",
    r"item8financial statements and supplementary data",
    r"item 8: financial statements and supplementary data",
    r"item 8:financial statements and supplementary data",
    r"item8: financial statements and supplementary data",
    r"item8:financial statements and supplementary data"
]

    item8_patterns = [
    r"item 8\. financial statements and supplementary data",
    r"item 8\.financial statements and supplementary data",
    r"item8\. financial statements and supplementary data",
    r"item8\.financial statements and supplementary data",
    r"item 8 financial statements and supplementary data",
    r"item 8financial statements and supplementary data",
    r"item8 financial statements and supplementary data",
    r"item8financial statements and supplementary data",
    r"item 8: financial statements and supplementary data",
    r"item 8:financial statements and supplementary data",
    r"item8: financial statements and supplementary data",
    r"item8:financial statements and supplementary data"
]


    # end_pattern_3q = r"ITEM 3\. QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK"


    ct = 0

    if filing_type == "10-Q":
      for pattern in item2_patterns:
          # Modify the pattern to ignore spaces/tabs
          adjusted_pattern = re.sub(r"\s+", r"\\s*", pattern)
          last_match = None

          for match in re.finditer(adjusted_pattern, content, re.IGNORECASE):
              #print(match)
              last_match = match  # Keep updating to the last match

          if last_match:
              start_index = last_match.end()

              for end_pattern in end_pattern_3q:
                  # Modify the end pattern similarly to ignore spaces/tabs
                  adjusted_end_pattern = re.sub(r"\s+", r"\\s*", end_pattern)
                  end_match = re.search(adjusted_end_pattern, content[start_index:], re.IGNORECASE)

                  if end_match:
                      end_index = start_index + end_match.start()
                      if end_index - start_index > 600:


                        return content[start_index:end_index]

              # If no end pattern is matched, return the content till the end

      return content[start_index:]

    else :
        for pattern in item7_patterns:
          # Modify the pattern to ignore spaces/tabs
          #print(pattern)
          adjusted_pattern = re.sub(r"\s+", r"\\s*", pattern)
          last_match = None

          for match in re.finditer(adjusted_pattern, content, re.IGNORECASE):
              last_match = match  # Keep updating to the last match

          if last_match:
              start_index = last_match.end()

              for end_pattern in item7a_patterns :
                  # Modify the end pattern similarly to ignore spaces/tabs
                  adjusted_end_pattern = re.sub(r"\s+", r"\\s*", end_pattern)
                  end_match = re.search(adjusted_end_pattern, content[start_index:], re.IGNORECASE)

                  if end_match:
                      end_index = start_index + end_match.start()
                      if end_index - start_index > 600:


                        return content[start_index:end_index]

              # If no end pattern is matched, return the content till the end
        return content[start_index:]

    return None

# Example usage


In [27]:
import os
import requests
from bs4 import BeautifulSoup
import re

headers = {'User-Agent': 'Your Name yourname@email.com'}

def get_extracted_data(stock, date):
    [m, d, y] = date.split("/")
    m = int(m)

    filing_type = ""

    if m >= 1 and m <= 3:
        filing_type = "10-Q"
    elif m >= 4 and m <= 6:
        filing_type = "10-Q"
    elif m >= 7 and m <= 9:
        filing_type = "10-Q"
    else:
        filing_type = "10-K"

    download_data(stock, date)

    filing_dir = os.path.join('sec-edgar-filings', stock, filing_type)

    if os.path.exists(filing_dir):
        for filename in os.listdir(filing_dir):
            url = f"https://www.sec.gov/Archives/edgar/data/{get_cik(stock)}/{remove_hifen(filename)}/{filename}-index.html"

            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, 'lxml')
            arr = soup.find_all('a')

            temp = str(arr[9])
            res = get_name(temp)
            new_url = f"https://www.sec.gov/{res}"
            print(new_url)
            response = requests.get(f"https://www.sec.gov/{res}", headers=headers)
            content = clean_xbrl(response.text)

            # with open("re.txt","w") as f:
            #   f.write(content)

            #filing_type = "10-Q"  # or "10-Q" depending on the date
            mda_content = "management and discussion analysis " + extract_mda(content, filing_type)


            folder_path = os.path.join('sec-edgar-filings', stock)
            delete_folder(folder_path)

            if mda_content:
                return mda_content
                pass
            else:
                print("MD&A section not found.")


    else:
        return "Data Not Available in SEC-Filings"

# get_extracted_data("HON" , "4/29/2014")


In [35]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/corporate_rating.csv')

i = 0
for i in range(2030):
   try:
    data.loc[i,"MDA Data"] = get_extracted_data(data.loc[i,"Symbol"],data.loc[i,"Date"])
    print(f"data from row{i} is extracted ")
   except  Exception as e:
    print(f"error on row{i} {e}")
    data.loc[i,"MDA Data"] = "Data Not Available"
    print(f"data from row{i} is not extracted ")

data

Streaming output truncated to the last 5000 lines.
https://www.sec.gov//Archives/edgar/data/1534701/000153470116000153/psx-2016630_10q.htm
Folder 'sec-edgar-filings/PSX' and its contents have been deleted.
data from row131 is extracted 
https://www.sec.gov//Archives/edgar/data/1534701/000153470114000116/psx-2014930_10q.htm
Folder 'sec-edgar-filings/PSX' and its contents have been deleted.
data from row132 is extracted 
https://www.sec.gov//Archives/edgar/data/1534701/000153470113000016/psx-6302013x10q.htm
Folder 'sec-edgar-filings/PSX' and its contents have been deleted.
data from row133 is extracted 
https://www.sec.gov//Archives/edgar/data/33185/000114420413023924/v339322_10q.htm
Folder 'sec-edgar-filings/EFX' and its contents have been deleted.
data from row134 is extracted 
https://www.sec.gov//Archives/edgar/data/33185/000114420414044723/v383655_10q.htm
Folder 'sec-edgar-filings/EFX' and its contents have been deleted.
data from row135 is extracted 
https://www.sec.gov//Archives/e

,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,freeCashFlowOperatingCashFlowRatio,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,MDA Data
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,0.437551,6.810673,9.809403,4.008012,0.049351,7.057088,15.565438,0.058638,3.906655,management and discussion analysis OF FINANCI...
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,0.541997,8.625473,17.402270,3.156783,0.048857,6.460618,15.914250,0.067239,4.002846,management and discussion analysis OF FINANCI...
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,0.513185,9.693487,13.103448,4.094575,0.044334,10.491970,18.888889,0.074426,3.483510,management and discussion analysis OF FINANCI...
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,-0.147170,-1.015625,14.440104,3.630950,-0.012858,4.080741,6.901042,0.028394,4.581150,management and discussion analysis OF FINANCI...
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,0.451372,7.135348,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790,management and discussion analysis OF FINANCI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,BB,Kaiser Aluminum Corporation,KALU,Standard & Poor's Ratings Services,4/28/2016,Capital Goods,2.962788,1.294743,0.428234,30.602414,...,0.602645,5.563630,5.958956,1.614282,-0.267117,-4.729967,9.232021,0.114089,14.542373,management and discussion analysis of Financi...
2026,B,Cresud S.A.C.I.F. y A.,CRESY,Fitch Ratings,11/30/2012,Finance,0.883875,0.842553,0.233830,147.599371,...,0.926665,1.459801,1.056480,4.034952,0.265092,9.358311,1.575328,0.283634,2.300168,Data Not Available in SEC-Filings
2027,B,Cresud S.A.C.I.F. y A.,CRESY,Fitch Ratings,6/15/2012,Finance,0.911713,0.748356,0.310640,131.644566,...,0.692804,0.744377,1.470201,3.825856,0.130692,22.440102,1.074441,0.217783,1.997608,Data Not Available in SEC-Filings
2028,CCC,Cresud S.A.C.I.F. y A.,CRESY,Fitch Ratings,8/1/2014,Finance,1.085007,1.026375,0.203490,151.660513,...,1.101462,2.487817,6.109814,3.939161,0.302997,9.604061,2.258650,0.252606,1.865682,Data Not Available in SEC-Filings


In [20]:
data[0:50]

,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,freeCashFlowOperatingCashFlowRatio,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,MDA Data
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,0.437551,6.810673,9.809403,4.008012,0.049351,7.057088,15.565438,0.058638,3.906655,OF FINANCIAL CONDITION AND RESULTS OF OPERATI...
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,0.541997,8.625473,17.402270,3.156783,0.048857,6.460618,15.914250,0.067239,4.002846,OF FINANCIAL CONDITION AND RESULTS OF OPERATI...
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,0.513185,9.693487,13.103448,4.094575,0.044334,10.491970,18.888889,0.074426,3.483510,OF FINANCIAL CONDITION AND RESULTS OF OPERATI...
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,-0.147170,-1.015625,14.440104,3.630950,-0.012858,4.080741,6.901042,0.028394,4.581150,OF FINANCIAL CONDITION AND RESULTS OF OPERATI...
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,0.451372,7.135348,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790,OF FINANCIAL CONDITION AND RESULTS OF OPERATI...
5,AA,Schlumberger N.V.,SLB,Egan-Jones Ratings Company,7/18/2014,Energy,1.741958,1.317156,0.220796,83.931968,...,0.644841,5.574517,5.792278,1.767609,0.157246,9.833295,8.644788,0.230445,4.044776,of Financial Condition and Results of Operati...
6,AA,Schlumberger N.V.,SLB,Egan-Jones Ratings Company,5/13/2014,Energy,1.939002,1.468909,0.256710,92.705452,...,0.597160,4.417989,6.326531,1.700068,0.151173,12.140376,7.398337,0.216233,3.998076,of Financial Condition and Results of Operati...
7,A,Schlumberger N.V.,SLB,Egan-Jones Ratings Company,7/23/2015,Energy,1.905814,1.544791,0.197791,90.336857,...,0.726292,5.047356,10.287293,1.908484,0.081212,15.264562,6.949487,0.248203,3.665200,of Financial Condition and Results of Operati...
8,AA,Honeywell International Inc.,HON,Egan-Jones Ratings Company,4/29/2014,Capital Goods,1.492419,1.129822,0.452859,74.102804,...,0.781546,4.308240,10.291200,2.601191,0.138574,10.897998,5.512462,0.110997,5.482026,OF FINANCIAL CONDITION AND RESULTS OF OPERATI...
9,A,Honeywell International Inc.,HON,Moody's Investors Service,2/16/2016,Capital Goods,1.091557,0.850961,0.296935,76.394469,...,0.803264,5.618107,9.692229,2.697369,0.170706,10.362903,6.994101,0.141365,4.793369,OF FINANCIAL CONDITION AND RESULTS OF OPERATI...


In [36]:
data.to_csv("generated_dataset.csv" , index=False)